## fillna problem 

`pandas.DataFrame` has a `fillna` function which is highly useful for as a compliment to outer joins. `docplex` doesn't play as nicely as it could with this function, because it won't allow a `Var` to be the argument to `fillna`.

This isn't particularly relevant because `docplex` doesn't have the toehold problem. It's mostly an academic problem for `docplex`.

In [1]:
from docplex.mp.model import Model
import  pandas as pd
m = Model()
v = m.continuous_var()
df = pd.DataFrame({"a":[1, 2, 3], 
                   "b":[4, 5, 6], 
                   "data":[1, 12, 19]})
df.set_index(["a", "b"], inplace=True)
df2 = pd.DataFrame({"a":[1, 2, 5], "b" : [4, 5, 2], "data":[1.1, 12.1, 19]})
df2.set_index(["a", "b"], inplace=True)

Ok, set up finished, here is the buggy step.

In [2]:
df.join(df2, how="outer", rsuffix="_2").fillna(v)

DOcplexException: Model<docplex_model1> did not solve successfully

This is a particularly weird and cryptic error message. I'm inclined to think of this as something `docplex` can fix. Note how I can combine `fillna` with new style classes just fine. (I.e. see below).

In [3]:
class MyClass(object):
    def __repr__(self):
        return "<I made this class>"
mc = MyClass()

In [4]:
df.join(df2, how="outer", rsuffix="_2").fillna(mc)

,,data,data_2
a,b,,
1,4,1,1.1
2,5,12,12.1
3,6,19,<I made this class>
5,2,<I made this class>,19


If I really did want to use a `Var` as an argument to `fillna`, it would be tough, since there is no easy workaround that I can see.

In [5]:
df.join(df2, how="outer", rsuffix="_2").fillna(2*v-v)

ValueError: invalid fill value with a <class 'docplex.mp.linear.LinearExpr'>

However, the toehold problem doesn't exist here in `docplex`, so there is no great worry.